In [1]:
import os
from pathlib import Path
from gurobipy import Model, GRB
from data_loader import load_prices, load_storage, load_efficiency, load_plant_capacity, load_demand, load_params

# Parameter path
current_dir = Path().resolve()
parameter_file = os.path.join(current_dir, "params.csv")
# Data Loading from data folder in the parent directory
data_folder = os.path.join(current_dir.parent, "data")
fuel_prices_file = os.path.join(data_folder, "fuel_prices.csv")           # Price in EUR/tonne (coal,oil) or EUR/m3 (gas)
storage_file = os.path.join(data_folder, "storage.csv")         # Max storage in tonnes/m3
efficiency_file = os.path.join(data_folder, "efficiency.csv")   # efficiency of MWh th to MWh el
plant_file = os.path.join(data_folder, "plant_capacity.csv")    # Plant capacity in MWh/month
demand_file = os.path.join(data_folder, "electricity_demand.csv")           # Monthly demand in MWh

# Load parameters
try:
    params_dict = load_params(parameter_file)
except Exception as e:
    raise RuntimeError(f"Error loading parameters: {e}")

start_date = params_dict.get("start_date", "2020-01")
end_date = params_dict.get("end_date", "2022-12")
supply_factor = params_dict.get("supply_factor", 0.6)
zone = params_dict.get("zone", "DK_1")
resample_method = params_dict.get("resample_method", "M")
# Get every row starting with 'fuel' as fuel types
fuels = [value for key, value in params_dict.items() if key.startswith("fuel")]

try:
    fuel_prices_df = load_prices(fuel_prices_file, start_date, end_date, resample_method= resample_method)
    storage = load_storage(storage_file)
    efficiency = load_efficiency(efficiency_file)
    plant_capacity = load_plant_capacity(plant_file)
    demand_df = load_demand(demand_file, start_date, end_date, resample_method= resample_method ,zone=zone ,supply_factor=supply_factor)
except Exception as e:
    raise RuntimeError(f"Error loading data: {e}")

# periods as Month PeriodIndex (so days_in_month works)
periods = demand_df.index.to_period('M')

print( periods)



# Model Initialization

m = Model("Fuel_Procurement_OneMonth")

# Decision Variables
x = {fuel: m.addVar(lb=0, ub=storage[fuel], name=f"buy_{fuel}") for fuel in fuel_prices_df.columns if fuel in fuels}
#y = {fuel: m.addVar(lb=0, ub=plant_capacity[fuel], name=f"gen_{fuel}") for fuel in fuel_prices_df.columns if fuel in fuels}

y = {}
for fuel in fuels:
                        
        hours_in_month = hours_in_month = (periods.days_in_month.to_series() * 24).sum()
        print(f"Hours in month {periods}: {hours_in_month}")

        # plant_capacity[fuel] is MW -> convert to MWh/month
        monthly_capacity_mwh = plant_capacity.get(fuel, 0) * hours_in_month
        y[fuel] = m.addVar(lb=0, ub=monthly_capacity_mwh, name=f"use_{fuel}")

m.update()



# Constraints
m.addConstr(sum(y[fuel] for fuel in y) >= demand_df.iloc[0][zone], "Demand_Constraint")
for fuel in y:
    m.addConstr(y[fuel] <= efficiency[fuel] * x[fuel], f"efficiency_Constraint_{fuel}")
    # Print the added constraint for verification
    print(f"Added constraint: gen_{fuel} <= {efficiency[fuel]} * buy_{fuel}")
m.update()

# Objective
m.setObjective(sum(fuel_prices_df.iloc[0][fuel] * x[fuel] for fuel in x),GRB.MINIMIZE) 
m.update()

# Optimize
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    for fuel in x:
        print(f"Purchased {x[fuel].X} units of {fuel}")
    for fuel in y:
        print(f"Generated {y[fuel].X} units of electricity from {fuel}")
    for fuel in x:
        cost = fuel_prices_df.iloc[0][fuel] * x[fuel].X
        print(f"Cost for {fuel}: EUR {cost}")
    total_cost = m.objVal
    print(f"Total Cost: EUR {total_cost}")
else:
    print("No optimal solution found.")


PeriodIndex(['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
             '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
             '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
             '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
             '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
             '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12'],
            dtype='period[M]', name='utc_timestamp')
Set parameter Username
Set parameter LicenseID to value 2728218
Academic license - for non-commercial use only - expires 2026-10-27
Hours in month PeriodIndex(['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
             '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
             '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
             '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
         